In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output
import random

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

from google.colab import drive, files
drive.mount('/content/drive')

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
features = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tennis Project/features_processed.csv')
features.columns

Index(['surface', 'A_rank', 'B_rank', 'A_elo_surface', 'B_elo_surface',
       'A_elo', 'B_elo', 'A_avged_elo', 'B_avged_elo', 'A_wins', 'B_wins',
       'A_wins_surface', 'B_wins_surface', 'A_12month_form', 'B_12month_form',
       'A_12month_form_surface', 'B_12month_form_surface', 'A_hand', 'B_hand'],
      dtype='object')

In [27]:
# Randomly assign winner and loser to player A and B, whilst maintaining attributes

features['A_won'] = 0
np.random.seed(seed=42)

for i in range(len(features)):
    playerA_won = np.random.randint(0, 2)
    features.loc[i, 'A_won'] = playerA_won
    if playerA_won == 0:
        features.loc[i, 'A_rank'], features.loc[i, 'B_rank'] = features.loc[i, 'B_rank'], features.loc[i, 'A_rank']
        features.loc[i, 'A_elo'], features.loc[i, 'B_elo'] = features.loc[i, 'B_elo'], features.loc[i, 'A_elo']
        features.loc[i, 'A_elo_surface'], features.loc[i, 'B_elo_surface'] = features.loc[i, 'B_elo_surface'], features.loc[i, 'A_elo_surface']
        features.loc[i, 'A_wins'], features.loc[i, 'B_wins'] = features.loc[i, 'B_wins'], features.loc[i, 'A_wins']
        features.loc[i, 'A_wins_surface'], features.loc[i, 'B_wins_surface'] = features.loc[i, 'B_wins_surface'], features.loc[i, 'A_wins_surface']
        features.loc[i, 'A_12month_form'], features.loc[i, 'B_12month_form'] = features.loc[i, 'B_12month_form'], features.loc[i, 'A_12month_form']
        features.loc[i, 'A_12month_form_surface'], features.loc[i, 'B_12month_form_surface'] =  features.loc[i, 'B_12month_form_surface'], features.loc[i, 'A_12month_form_surface']
        features.loc[i, 'A_hand'], features.loc[i, 'B_hand'] = features.loc[i, 'B_hand'], features.loc[i, 'A_hand']

In [93]:
features = pd.get_dummies(features)

In [95]:
features = features.dropna().reset_index(drop=True)
features

,A_rank,B_rank,A_elo_surface,B_elo_surface,A_elo,B_elo,A_avged_elo,B_avged_elo,A_wins,B_wins,A_wins_surface,B_wins_surface,A_12month_form,B_12month_form,A_12month_form_surface,B_12month_form_surface,A_hand,B_hand,A_won,surface_carpet_False,surface_carpet_True,surface_clay_False,surface_clay_True,surface_grass_False,surface_grass_True,surface_hard_False,surface_hard_True
0,3.0,23.0,1988.523,1644.770,2151.654,1772.521,2070.0885,1708.6455,0.0,0.0,0.0,0.0,0.833333,0.000000,0.833333,0.000000,0.0,0.0,0.0,True,False,True,False,False,True,True,False
1,30.0,22.0,1501.014,1584.360,1714.681,1859.956,1722.1580,1607.8475,0.0,0.0,0.0,0.0,0.000000,0.666667,0.000000,0.666667,0.0,1.0,1.0,True,False,True,False,False,True,True,False
2,55.0,58.0,1554.535,1792.652,1712.486,1821.925,1633.5105,1807.2885,0.0,0.0,0.0,0.0,0.500000,0.000000,0.500000,0.000000,0.0,0.0,0.0,True,False,True,False,False,True,True,False
3,15.0,39.0,1740.677,1542.475,1885.650,1794.640,1813.1635,1668.5575,0.0,0.0,0.0,0.0,0.750000,0.000000,0.750000,0.000000,0.0,0.0,0.0,True,False,True,False,False,True,True,False
4,18.0,97.0,1766.935,1503.159,1922.537,1607.397,1844.7360,1555.2780,0.0,0.0,0.0,0.0,0.666667,0.000000,0.666667,0.000000,1.0,0.0,0.0,True,False,True,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143489,56.0,7.0,1590.551,1901.149,1684.596,1926.081,1913.6150,1637.5735,0.0,0.0,0.0,0.0,0.515152,0.717949,0.533333,0.783784,0.0,0.0,0.0,True,False,False,True,True,False,True,False
143490,47.0,29.0,1672.773,1694.263,1698.955,1724.848,1709.5555,1685.8640,0.0,0.0,0.0,0.0,0.680000,0.525424,0.708333,0.617647,0.0,0.0,0.0,True,False,False,True,True,False,True,False
143491,19.0,117.0,1554.270,1542.649,1797.649,1567.658,1555.1535,1675.9595,1.0,0.0,0.0,0.0,0.603448,0.700000,0.500000,0.833333,0.0,0.0,0.0,True,False,False,True,True,False,True,False
143492,44.0,7.0,1605.691,1905.735,1796.400,1932.462,1919.0985,1701.0455,0.0,1.0,0.0,0.0,0.636364,0.717949,0.666667,0.783784,0.0,0.0,0.0,True,False,False,True,True,False,True,False


In [100]:
X = features.drop('A_won', axis=1)
y = features['A_won']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [32]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7727098505174397


In [33]:
def model_accuracy(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [42]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }

model = GridSearchCV(LogisticRegression(penalty='l2'), param_grid)

In [43]:
model_accuracy(model)

0.7727098505174397

In [47]:
cross_val_score(model, X_train, y_train, cv=5).mean()

0.773108584868679

In [114]:
model.coef_

array([[ 6.15968880e-02, -5.80097250e-02,  5.40997917e-01,
        -5.30841466e-01, -2.20639864e-01,  2.42943303e-01,
         7.45879767e-03,  1.03202710e-02, -8.76670757e-02,
         6.60231158e-02,  8.85415739e-03,  7.26587660e-03,
        -1.55220664e-01,  1.34256444e-01, -1.61261804e+00,
         1.59611913e+00, -1.20230306e-02,  5.81273225e-03,
        -4.88505872e-03,  4.88505872e-03,  8.85770306e-03,
        -8.85770306e-03, -7.82631743e-03,  7.82631743e-03,
        -6.12021786e-04,  6.12021786e-04]])

In [97]:
# Predicting outcome based only on ranking
correct = 0
for i in range(len(features)):
    if features.loc[i, 'A_rank'] > features.loc[i, 'B_rank'] and features.loc[i, 'A_won'] == 1:
        correct += 1
    elif features.loc[i, 'A_rank'] < features.loc[i, 'B_rank'] and features.loc[i, 'A_won'] == 0:
        correct += 1
    else:
        pass

print('Accuracy: ', correct/len(features))

Accuracy:  0.6568358258881904


In [98]:
# Predicting outcome based only on Elo
correct = 0
for i in range(len(features)):
    if features.loc[i, 'A_elo'] > features.loc[i, 'B_elo'] and features.loc[i, 'A_won'] == 1:
        correct += 1
    elif features.loc[i, 'A_elo'] < features.loc[i, 'B_elo'] and features.loc[i, 'A_won'] == 0:
        correct += 1
    else:
        pass

print('Accuracy: ', correct/len(features))

Accuracy:  0.3364670299803476


In [115]:
# Predicting outcome based only on 12 month form
correct = 0
for i in range(len(features)):
    if features.loc[i, 'A_12month_form'] > features.loc[i, 'B_12month_form'] and features.loc[i, 'A_won'] == 1:
        correct += 1
    elif features.loc[i, 'A_12month_form'] < features.loc[i, 'B_12month_form'] and features.loc[i, 'A_won'] == 0:
        correct += 1
    else:
        pass

print('Accuracy: ', correct/len(features))

Accuracy:  0.27714747654954214
